In [1]:
%run 10_ngram_common.ipynb

/work/van-speech-nlp/jindaznb/asrenv/lib/python3.10/site-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


In [2]:
normal_speakers = ['MC01', 'MC02', 'MC03', 'MC04','FC01','FC02','FC03']
atypical_speakers = ['F01', 'F03', 'F04', 'M01', 'M02', 'M03', 'M04', 'M05']

# read europarl

In [3]:
europarl = load_dataset(f"{username}/{source_lang}_corpora_parliament_processed", split="train",cache_dir="/work/van-speech-nlp/cache")
translations=europarl['text']

In [4]:
len(translations)

2051014

In [6]:
# import nltk
# total_tokens = 0

# # Iterate over each sentence in the translations corpus
# for sentence in translations:
#     # Tokenize the sentence into individual words
#     tokens = nltk.word_tokenize(sentence)
#     # Add the number of tokens in the current sentence to the total count
#     total_tokens += len(tokens)

# # Print the total number of tokens in the translations corpus
# print("Total number of tokens in corpus:", total_tokens)

In [7]:
word_set_europarl = set()

for sentence in translations:
    words = sentence.split()
    word_set_europarl.update(words)

In [12]:
len(word_set_europarl)

120001

In [9]:
filtered_word_set_cleaned = set()

for word in translations:
    if word:
        cleaned_word = re.sub(r'[^a-zA-Z0-9]', '', word.lower())
        filtered_word_set_cleaned.add(cleaned_word)

cleaned_word_count = len(filtered_word_set_cleaned)

In [11]:
cleaned_word_count

1994288

# read torgo

In [13]:
pattern = "keep_all"

In [13]:
# Read the dataset
data_df = pd.read_csv('torgo_new.csv')
dataset_csv = load_dataset('csv', data_files='torgo_new.csv')

speakers = data_df['speaker_id'].unique()

print(f'Speakers: {", ".join(speakers)}')

Speakers: F01, F03, F04, FC01, FC02, FC03, M01, M02, M03, M04, M05, MC01, MC02, MC03, MC04


In [14]:
import re

# Define a function to clean the text
def clean_text(text):
    # Remove non-alphanumeric characters and convert to lowercase
    cleaned_text = ' '.join(re.sub(r'[^a-zA-Z0-9]', '', word.lower()) for word in text.split())
    return cleaned_text

# Apply the clean_text function to the 'text' column in data_df
data_df['text'] = data_df['text'].apply(clean_text)

In [15]:
from collections import defaultdict

# Initialize a defaultdict to store the texts for each speaker
speaker_texts = defaultdict(list)

# Iterate through each test speaker
for test_speaker in atypical_speakers:
    valid_speaker = 'F03' if test_speaker != 'F03' else 'F04'
    train_speaker = [s for s in speakers if s not in [test_speaker, valid_speaker]]
    torgo_dataset = DatasetDict()
    torgo_dataset['train'] = dataset_csv['train'].filter(lambda x: x in train_speaker, input_columns=['speaker_id'])
    torgo_dataset['validation'] = dataset_csv['train'].filter(lambda x: x == valid_speaker, input_columns=['speaker_id'])
    torgo_dataset['test'] = dataset_csv['train'].filter(lambda x: x == test_speaker, input_columns=['speaker_id'])

    torgo_dataset['train'] = torgo_dataset['train'].filter(lambda x: x['test_data'] == 0)
    torgo_dataset['validation'] = torgo_dataset['validation'].filter(lambda x: x['test_data'] == 0)
    torgo_dataset['test'] = torgo_dataset['test'].filter(lambda x: x['test_data'] == 1)

    # Filter the DataFrame to get the data for the train split for the selected train speakers
    test_data = torgo_dataset['test']
    
    # Append the texts from the filtered DataFrame to the corresponding speaker in the speaker_texts_train dictionary
    for text in test_data['text']:
        # print(text)
        new_text=clean_text(text)
        # print(new_text)
        speaker_texts[test_speaker].append(new_text)

# # Print the number of texts collected for each speaker
for speaker_id, texts in speaker_texts.items():
    print(f"{speaker_id}: {len(texts)}")
    for text in texts[:10]:
        print(text)
    print()

F01: 126
stick
except in the winter when the ooze or snow or ice prevents
up
know
he slowly takes a short walk in the open air each day
double
you wished to know all about my grandfather
knee
she had your dark suit in greasy wash water all year
car

F03: 592
beta
stubble
stubble
trace
goat
she had your dark suit in greasy wash water all year
well he is nearly ninetythree years old
knew
sip
alpha

F04: 367
fee
yet he still thinks as swiftly as ever
knew
trait
stubble
you wished to know all about my grandfather
chair
trace
go
hair

M01: 407
when he speaks his voice is just a bit cracked and quivers a trifle
trait
trouble
fee
raid
except in the winter when the ooze or snow or ice prevents
a long flowing beard clings to his chin
a long flowing beard clings to his chin
fair
bit

M02: 423
alpha
sip
dagger
fair
park
bubble
bit
well he is nearly ninetythree years old
trouble
chair

M03: 442
now
know
gadget
race
well he is nearly ninetythree years old
well he is nearly ninetythree years old
hai

# calculate unique words

In [16]:
unique_words_per_speaker = {}

for speaker_id, texts in speaker_texts.items():
    unique_words = set()
    for text in texts:
        words = text.split()
        for word in words:
            cleaned_word = re.sub(r'[^a-zA-Z0-9]', '', word.lower())
            if cleaned_word:  
                unique_words.add(cleaned_word) 
    unique_words_per_speaker[speaker_id] = unique_words

for speaker_id, unique_words in unique_words_per_speaker.items():
    print(f"{speaker_id}: {len(unique_words)} unique words")

F01: 117 unique words
F03: 447 unique words
F04: 420 unique words
M01: 381 unique words
M02: 392 unique words
M03: 399 unique words
M04: 283 unique words
M05: 436 unique words


# vocabulary similarity

In [17]:
out_of_domain_words_per_speaker = {}

for speaker_id, unique_words in unique_words_per_speaker.items():
    overlap = len(unique_words & word_set_europarl)
    similarity = overlap / len(unique_words) if len(unique_words) > 0 else 0
    out_of_domain_similarity = (1 - similarity)*100

    out_of_domain_words = {word for word in unique_words if word not in word_set_europarl}
    out_of_domain_words_per_speaker[speaker_id] = out_of_domain_words

    print(f"{speaker_id}: {out_of_domain_similarity:.2f}% dissimilarity with word_set_europarl")
    print(f"{speaker_id}: out_of_domain_words count: {len(out_of_domain_words)}")
    print(f"Out of domain words for {speaker_id}: {out_of_domain_words}")
    print()

F01: 2.56% dissimilarity with word_set_europarl
F01: out_of_domain_words count: 3
Out of domain words for F01: {'quivers', 'stubble', 'ooze'}

F03: 4.92% dissimilarity with word_set_europarl
F03: out_of_domain_words count: 22
Out of domain words for F03: {'woolen', 'dart', 'slicker', 'nancys', 'toot', 'youd', 'stubble', 'mut', 'fleecy', 'glitter', 'schoolhouse', 'ooze', 'hem', 'sitter', 'cupping', 'shimmers', 'quivers', 'advisement', 'droop', 'yell', 'briar', 'youre'}

F04: 5.00% dissimilarity with word_set_europarl
F04: out_of_domain_words count: 21
Out of domain words for F04: {'woolen', 'dart', 'slicker', 'nancys', 'toot', 'youd', 'stubble', 'mut', 'fleecy', 'frock', 'schoolhouse', 'ooze', 'hem', 'shimmers', 'quivers', 'advisement', 'droop', 'yell', 'briar', 'youre', 'lilyrare'}

M01: 4.72% dissimilarity with word_set_europarl
M01: out_of_domain_words count: 18
Out of domain words for M01: {'woolen', 'stubble', 'dart', 'quivers', 'nancys', 'mut', 'advisement', 'fleecy', 'droop', 'ye

# Out of domain analysis

In [19]:
import kenlm

model=kenlm.Model("europarl_lm_arpa_files/3gram.bin") 
per=model.perplexity("your text sentance")
print(per)

per=model.perplexity("resumption of the session")
print(per)

2445.298548730424
13.064232719071327


In [21]:
# Initialize a dictionary to store perplexities per speaker
perplexities_per_speaker = {}

# Calculate perplexity for each speaker
for speaker_id, texts in cleaned_speaker_texts.items():
    perplexities = []
    for text in texts:
        perplexity = model.perplexity(text)
        perplexities.append(perplexity)
    avg_perplexity = np.mean(perplexities)
    perplexities_per_speaker[speaker_id] = avg_perplexity

# Print average perplexity for each speaker
for speaker_id, avg_perplexity in perplexities_per_speaker.items():
    print(f"Speaker {speaker_id}: Average Perplexity: {avg_perplexity}")

Speaker F01: Average Perplexity: 928.958100079841
Speaker F03: Average Perplexity: 1609.073922518608
Speaker F04: Average Perplexity: 1663.791194970111
Speaker M01: Average Perplexity: 1618.6761751822528
Speaker M02: Average Perplexity: 1562.3229905124986
Speaker M03: Average Perplexity: 1526.6267614069698
Speaker M04: Average Perplexity: 1498.6719645933758
Speaker M05: Average Perplexity: 1431.2883772418654


# Average Perplexity: 1498.1788760387824 showing that TORGO is out of domain text

# clean the text

In [58]:
import re

cleaned_speaker_texts = {}

for speaker_id, texts in speaker_texts.items():
    cleaned_texts = []
    for text in texts:
        cleaned_text = ' '.join(re.sub(r'[^a-zA-Z0-9]', '', word.lower()) for word in text.split())
        cleaned_texts.append(cleaned_text)
    cleaned_speaker_texts[speaker_id] = cleaned_texts

for speaker_id, texts in cleaned_speaker_texts.items():
    print(f"Speaker {speaker_id}:")
    for text in texts[:2]:  
        print(text)
    print()

Speaker F01:
stick
except in the winter when the ooze or snow or ice prevents

Speaker F03:
beta
stubble

Speaker F04:
fee
yet he still thinks as swiftly as ever

Speaker M01:
when he speaks his voice is just a bit cracked and quivers a trifle
trait

Speaker M02:
grow
feed

Speaker M03:
now
know

Speaker M04:
trouble
spark

Speaker M05:
swore
train



In [60]:
# Initialize a dictionary to store perplexities per speaker
perplexities_per_speaker = {}

# Calculate perplexity for each speaker
for speaker_id, texts in cleaned_speaker_texts.items():
    perplexities = []
    for text in texts:
        perplexity = model.perplexity(text)
        perplexities.append(perplexity)
    avg_perplexity = np.mean(perplexities)
    perplexities_per_speaker[speaker_id] = avg_perplexity

# Print average perplexity for each speaker
for speaker_id, avg_perplexity in perplexities_per_speaker.items():
    print(f"Speaker {speaker_id}: Average Perplexity: {avg_perplexity}")

Speaker F01: Average Perplexity: 1014.5739804044058
Speaker F03: Average Perplexity: 1468.7080678251675
Speaker F04: Average Perplexity: 1474.5388428766746
Speaker M01: Average Perplexity: 1534.7643705710645
Speaker M02: Average Perplexity: 1492.4557345179953
Speaker M03: Average Perplexity: 1492.9785709867167
Speaker M04: Average Perplexity: 1531.2270123869641
Speaker M05: Average Perplexity: 1584.9380105002526
